In [98]:
from crocoddyl import *
import pinocchio as pin
import numpy as np

robot = loadBorinotArm()
robot.initViewer(loadModel=True)

q0 = [-3.14, 3.14]
robot.q0.flat = q0
robot.framesForwardKinematics(robot.q0)
robot.display(robot.q0)

IDX_LINK1 = robot.model.getFrameId('link1', pin.FrameType.BODY)
IDX_LINK2 = robot.model.getFrameId('link2', pin.FrameType.BODY)
Mlink1 = robot.data.oMf[IDX_LINK1]
Mlink2 = robot.data.oMf[IDX_LINK2]

target_pos  = np.array([0,0,0.3])
target_quat = pin.Quaternion(1, 0, 0, 0)
target_quat.normalize()

Mref = pin.SE3()
Mref.translation = target_pos.reshape(3,1)
Mref.rotation = target_quat.matrix()

#robot.viewer.gui.addXYZaxis('world/framelink1', [1., 0., 0., 1.], .005, 0.05)
#robot.viewer.gui.addXYZaxis('world/framelink2', [1., 0., 0., 1.], .005, 0.05)
#robot.viewer.gui.addXYZaxis('world/frameref', [1., 0., 0., 1.], .005, 0.05)

#robot.viewer.gui.applyConfiguration('world/framelink1', pin.se3ToXYZQUATtuple(Mlink1))
#robot.viewer.gui.applyConfiguration('world/framelink2', pin.se3ToXYZQUATtuple(Mlink2))
#robot.viewer.gui.applyConfiguration('world/frameref', pin.se3ToXYZQUATtuple(Mref))

robot.viewer.gui.refresh()

In [111]:
state = StatePinocchio(robot.model)

goalOrientationCost1 = CostModelFrameRotation(robot.model, 
                                             frame=robot.model.getFrameId('link1'), 
                                             ref=Mref.rotation, 
                                             nu=robot.model.nv)
goalOrientationCost2 = CostModelFrameRotation(robot.model, 
                                             frame=robot.model.getFrameId('link2'), 
                                             ref=Mref.rotation, 
                                             nu=robot.model.nv)

xRegCost = CostModelState(robot.model, state, ref=state.zero(), nu=robot.model.nv)
uRegCost = CostModelControl(robot.model, nu=robot.model.nv)
uLimCost = CostModelControl(robot.model, 
                            nu=robot.model.nv,
                            activation = ActivationModelInequality(np.array([-1e-3, -1e-3]), np.array([1e-3, 1e-3])))



runningCostModel = CostModelSum(robot.model)
terminalCostModel = CostModelSum(robot.model)

runningCostModel.addCost(name="ori1", weight=1e-2, cost=goalOrientationCost1)
runningCostModel.addCost(name="ori2", weight=1e-2, cost=goalOrientationCost2)
runningCostModel.addCost(name="regx", weight=1e-7, cost=xRegCost)
runningCostModel.addCost(name="regu", weight=1e-1, cost=uRegCost)
runningCostModel.addCost(name="limu", weight=10000, cost=uLimCost)
terminalCostModel.addCost(name="ori1", weight=5, cost=goalOrientationCost1)
terminalCostModel.addCost(name="ori2", weight=5, cost=goalOrientationCost2)

In [112]:
runningModel = IntegratedActionModelEuler(DifferentialActionModelFullyActuated(robot.model, runningCostModel))
terminalModel = IntegratedActionModelEuler(DifferentialActionModelFullyActuated(robot.model, terminalCostModel))

# Defining the time duration for running action models and the terminal one
dt = 1e-2
runningModel.timeStep = dt

# For this optimal control problem, we define 250 knots (or running action
# models) plus a terminal knot
T = 100
q0 = [-1.57, 2]
x0 = np.hstack([q0, np.zeros(robot.model.nv)])
problem = ShootingProblem(x0, [runningModel] * T, terminalModel)

# Creating the DDP solver for this OC problem, defining a logger
ddp = SolverDDP(problem)
ddp.callback = [CallbackDDPVerbose()]
ddp.callback.append(CallbackDDPLogger())

# Solving it with the DDP algorithm
ddp.solve()

iter 	 cost 	      stop 	    grad 	  xreg 	      ureg 	 step 	 feas
   0  8.22722e+02  1.53608e-07  2.85796e-11  1.00000e-09  1.00000e-09   1.0000     1
   1  7.82866e+02  1.64964e+07  1.64366e+03  1.00000e-09  1.00000e-09   0.1250     1
   2  7.74377e+02  1.59686e+07  1.55226e+03  1.00000e-09  1.00000e-09   0.0156     1
   3  7.65439e+02  1.66619e+07  1.51114e+03  1.00000e-09  1.00000e-09   0.0156     1
   4  7.46728e+02  1.71219e+07  1.47746e+03  1.00000e-09  1.00000e-09   0.0625     1
   5  6.73463e+02  1.67466e+07  1.42448e+03  1.00000e-09  1.00000e-09   0.1250     1
   6  5.15004e+02  1.34686e+07  1.34587e+03  1.00000e-09  1.00000e-09   0.2500     1
   7  2.06038e+02  1.11691e+07  9.69961e+02  1.00000e-09  1.00000e-09   0.5000     1
   8  2.00761e+02  4.14994e+06  4.06760e+02  1.00000e-09  1.00000e-09   0.0312     1
   9  1.47211e+02  3.94685e+06  3.53679e+02  1.00000e-09  1.00000e-09   0.2500     1
iter 	 cost 	      stop 	    grad 	  xreg 	      ureg 	 step 	 feas
  10  1.45307e

iter 	 cost 	      stop 	    grad 	  xreg 	      ureg 	 step 	 feas
  90  9.68266e+00  3.39338e+03  1.86600e-01  1.00000e-06  1.00000e-06   0.0625     1
  91  9.67773e+00  5.36183e+03  3.10928e-01  1.00000e-06  1.00000e-06   0.0312     1
  92  9.66189e+00  2.90718e+03  1.72239e-01  1.00000e-06  1.00000e-06   0.2500     1
  93  9.64225e+00  2.91833e+04  1.69669e+00  1.00000e-06  1.00000e-06   0.0312     1
  94  9.63974e+00  3.81247e+03  2.09775e-01  1.00000e-06  1.00000e-06   0.0625     1
  95  9.63921e+00  7.92591e+03  4.91333e-01  1.00000e-05  1.00000e-05   0.0020     1
  96  9.63659e+00  6.15321e+03  3.70180e-01  1.00000e-05  1.00000e-05   0.0078     1
  97  9.63661e+00  4.43899e+03  2.76598e-01  1.00000e-04  1.00000e-04   0.0020     1
  98  9.63637e+00  3.23573e+03  1.68842e-01  1.00000e-03  1.00000e-03   0.0020     1
  99  9.60595e+00  2.74333e+03  5.91110e-02  1.00000e-04  1.00000e-04   1.0000     1


([array([-1.57,  2.  ,  0.  ,  0.  ]),
  array([-1.57729822,  2.25472028, -0.72982157, 25.47202793]),
  array([-1.58761746,  2.67904827, -1.03192475, 42.43279939]),
  array([-1.59731008,  3.25701976, -0.9692615 , 57.7971489 ]),
  array([-1.60946252,  4.05982748, -1.21524398, 80.28077163]),
  array([ -1.6522568 ,   5.23211205,  -4.27942802, 117.22845748]),
  array([ -1.77571909,   6.84244399, -12.34622953, 161.03319408]),
  array([-1.78004082,  7.21894508, -0.43217242, 37.65010873]),
  array([ -1.75504158,   6.92260739,   2.4999235 , -29.63376914]),
  array([ -1.70629477,   6.09817215,   4.87468119, -82.44352436]),
  array([  -1.62343745,    4.53268252,    8.2857316 , -156.54896297]),
  array([  -1.65617699,    2.43274086,   -3.27395349, -209.99416576]),
  array([  -1.51706154,    1.13198405,   13.91154471, -130.0756807 ]),
  array([  -1.33968906,    0.12781729,   17.73724874, -100.41667583]),
  array([ -1.18632898,  -0.57513708,  15.33600765, -70.29543771]),
  array([ -1.06266698,  -1.

In [108]:
from crocoddyl.diagnostic import displayTrajectory

displayTrajectory(robot, ddp.xs, runningModel.timeStep)

In [109]:
ddp.us

[array([ 0.03434323, -0.00174103]),
 array([ 0.03144137, -0.00107895]),
 array([0.02866946, 0.0004568 ]),
 array([2.60434255e-02, 5.76080344e-05]),
 array([ 2.35657120e-02, -9.79456185e-05]),
 array([ 2.12411161e-02, -8.49880785e-05]),
 array([ 1.90745430e-02, -1.25102029e-05]),
 array([1.70690156e-02, 7.25447062e-05]),
 array([0.01522504, 0.00014527]),
 array([0.01354063, 0.00019581]),
 array([0.01201112, 0.00022169]),
 array([0.01062994, 0.00022519]),
 array([0.009389  , 0.00021008]),
 array([0.0082792 , 0.00018091]),
 array([0.00729088, 0.00014233]),
 array([6.41421515e-03, 9.89299792e-05]),
 array([5.63949851e-03, 5.51275868e-05]),
 array([4.95734605e-03, 1.46062335e-05]),
 array([ 4.35882837e-03, -2.01188236e-05]),
 array([ 3.83555381e-03, -4.79079750e-05]),
 array([ 3.37971267e-03, -6.87669497e-05]),
 array([ 2.98409523e-03, -8.33785617e-05]),
 array([ 2.64209107e-03, -9.26838536e-05]),
 array([ 2.34767505e-03, -9.76317661e-05]),
 array([ 2.09538333e-03, -9.90719245e-05]),
 array